This notebook is for the purpose of converting the essays in string form to word embedding vectorized form. This notebook makes use of the GLoVe word embeddings. More information about these word embeddings can be found here: https://nlp.stanford.edu/projects/glove/

If you'd like to move directly to the Neural Network, you can skip this notebook completely and load the prepared data directly!

If you would like to follow along with the preprocessing work, follow the link above and download the glove.6B.zip file to the "data/" directory of this project. Then uncompress the zip file to reveal directory named "glove.6B/" with 4 txt files inside. These are the four available word embeddings. You are now able to proceed with this notebook. 

In [39]:
from src.preprocess import get_data, vectorize_essays, pad_embedding, word_count
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# Import training data from tsv file and store in a pandas dataframe
data_path = './data/training_set_rel3.tsv'
essay_df = get_data(data_path)

We can inspect the contents of the dataframe to understand what information is provided in the training set
More EDA if we have time

We'd like to drop some columns that are irrelevant to the neural network

In [41]:
drop_labels =['rater1_trait1', 'rater1_trait2', 'rater1_trait3', 'rater1_trait4', 'rater1_trait5', 'rater1_trait6',\
              'rater2_trait1', 'rater2_trait2', 'rater2_trait3', 'rater2_trait4', 'rater2_trait5', 'rater2_trait6',\
              'rater3_trait1', 'rater3_trait2', 'rater3_trait3', 'rater3_trait4', 'rater3_trait5', 'rater3_trait6',\
              'rater1_domain1', 'rater2_domain1', 'rater3_domain1', 'rater1_domain2', 'rater2_domain2', 'domain2_score']
essay_df.drop(columns=drop_labels, inplace=True)
essay_df.tail()

,essay_id,essay_set,essay,domain1_score
12971,21626,8,In most stories mothers and daughters are eit...,35
12972,21628,8,I never understood the meaning laughter is th...,32
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",40
12974,21630,8,Trippin' on fen...,40
12975,21633,8,Many people believe that laughter can improve...,40


The given dataset holds essays from 8 distinct "essay sets". In order to speed training time and reduce memory impact, we will only examine a subset of these essay sets. The rest of the essays can be discarded

In [42]:
sets = [1,3,4,5,6]
essay_df = essay_df[essay_df['essay_set'].isin(sets)]

print("Examining essays in sets {}".format(essay_df.essay_set.unique()))

Examining essays in sets [1 3 4 5 6]


In [43]:
essays = essay_df['essay'] # A list of essays as strings

# Add a column to dataframe with essays as word embeddings
essay_df['essays_embed'] = vectorize_essays(essays, embed_size=100, verbose=True)

The total number of essays is 8884


1000 Essays Vectorized


2000 Essays Vectorized


3000 Essays Vectorized


4000 Essays Vectorized


5000 Essays Vectorized


6000 Essays Vectorized


7000 Essays Vectorized


8000 Essays Vectorized


8884 Total Essays Vectorized!


In [44]:
essay_df['word_count'] = essay_df.apply(lambda row: word_count(row), axis=1)
# Potentially add matplotlib visualization later
print("*** Word Count Statistics ***\n")
#word_count_stats = [None]*len(sets)
for set in sets:
    print('*'*40)
    print("Word count statistics for set {}: ".format(set))
    print(essay_df[essay_df['essay_set']==set]['word_count'].describe())
    print('\n')

*** Word Count Statistics ***

****************************************
Word count statistics for set 1: 
count    1783.000000
mean      403.526640
std       133.734738
min        10.000000
25%       315.000000
50%       402.000000
75%       487.000000
max       918.000000
Name: word_count, dtype: float64


****************************************
Word count statistics for set 3: 
count    1726.000000
mean      118.953071
std        58.932136
min        10.000000
25%        73.000000
50%       110.000000
75%       159.000000
max       401.000000
Name: word_count, dtype: float64


****************************************
Word count statistics for set 4: 
count    1770.000000
mean      102.149153
std        57.075221
min         2.000000
25%        57.000000
50%        94.000000
75%       138.750000
max       401.000000
Name: word_count, dtype: float64


****************************************
Word count statistics for set 5: 
count    1805.000000
mean      134.165651
std        63.9813

We would also like to remove some outliers for word count. Those essays with length greater than 600 words will be removed.

In [45]:
max_length = 600 # Maximum length for an essay
essay_df = essay_df[essay_df['word_count']<=max_length]

print(essay_df['word_count'].describe())

count    8767.000000
mean      179.020532
std       125.991802
min         2.000000
25%        90.000000
50%       147.000000
75%       218.500000
max       600.000000
Name: word_count, dtype: float64


Next we will pad each essay to the maximum length so that all essays have a uniform size of word embedding matrix

In [46]:
essays_embed = essay_df['essays_embed']
essay_df['essays_embed'] = pad_embedding(essays_embed,\
                                         max_length=max_length, right_pad=True)

In [54]:
essay_df.iloc[835]

essay_id                                                       897
essay_set                                                        1
essay            Dear @ORGANIZATION1, The age of the computer i...
domain1_score                                                    9
essays_embed                                                  None
word_count                                                     600
Name: 894, dtype: object

In [55]:
essay_df.dropna(subset=['essays_embed'], inplace=True)

In [57]:
# Add max score for each set
score_ranges = [(0,0), (2,12), (0,0), (0,3), (0,3), (0,4), (0,4)]

for idx in essay_df.index:
    essay_df.loc[idx,'max_score']=score_ranges[essay_df.loc[idx,'essay_set']][1]

In [58]:
maximum_score = 12
for idx in essay_df.index:
    essay_df.loc[idx,'norm_score']=essay_df.loc[idx,'domain1_score']*maximum_score/essay_df.loc[idx,'max_score']

In [36]:
essay_df[essay_df['essay_set']==5].head()

,essay_id,essay_set,essay,domain1_score,essays_embed,word_count,max_score,norm_score
7079,11827,5,"In this memoir of Narciso Rodriguez, @PERSON3'...",2,"[[0.085703, -0.22201, 0.16569, 0.13373, 0.3823...",156,4.0,6.0
7080,11828,5,Throughout the excerpt from Home the Blueprint...,2,"[[-0.40822, 0.32473, -0.18726, 0.33417, 0.7855...",183,4.0,6.0
7081,11829,5,The mood the author created in the memoir is l...,3,"[[-0.038194, -0.24487, 0.72812, -0.39961, 0.08...",118,4.0,9.0
7082,11830,5,The mood created by the author is showing how ...,1,"[[-0.038194, -0.24487, 0.72812, -0.39961, 0.08...",74,4.0,3.0
7083,11831,5,The mood created in the memoir is happiness an...,3,"[[-0.038194, -0.24487, 0.72812, -0.39961, 0.08...",139,4.0,9.0


In [59]:
# Store the preprocessed data frame for later use
essay_df.to_pickle('./data/essay_df.pkl')